<a href="https://colab.research.google.com/github/zahrasadat9/DL-Adventures/blob/main/movielens_100_recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# libs and data

In [1]:
! pip install kaggle
# 2. Make a directory named “.kaggle”

! mkdir ~/.kaggle
# 3. Copy the “kaggle.json” into this new directory

! cp kaggle.json ~/.kaggle/
# 4. Allocate the required permission for this file.

! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! kaggle datasets download prajitdatta/movielens-100k-dataset

 84% 4.00M/4.77M [00:00<00:00, 6.25MB/s]
100% 4.77M/4.77M [00:00<00:00, 5.24MB/s]


In [3]:
!unzip "/content/movielens-100k-dataset.zip" ;

Archive:  /content/movielens-100k-dataset.zip
  inflating: ml-100k/README          
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from tabulate import tabulate

In [5]:
udata=pd.read_csv('/content/ml-100k/u.data',delimiter='\t',names=['user id', 'item id', 'rating', 'timestamp'])

In [ ]:
udata.head(5)

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
names="movie id | movie title | release date | video release date |IMDb URL | unknown | Action | Adventure | Animation |Children | Comedy | Crime | Documentary | Drama | Fantasy |Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |Thriller | War | Western "
names=names.replace(" ", "").split('|')
# names

In [7]:
uitem=pd.read_csv('/content/ml-100k/u.item',delimiter='|', encoding="iso-8859-1",names=names)#,on_bad_lines='skip')


In [8]:
uitem=uitem.rename({'movieid': 'item id'},axis='columns')
uitem.head(5)

,item id,movietitle,releasedate,videoreleasedate,IMDbURL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
uitem2= uitem[['item id','movietitle']]
uitem2.head(2)

,item id,movietitle
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


In [10]:
udata2=pd.merge(uitem2,udata,on="item id")
udata2.head(2)

,item id,movietitle,user id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088


In [11]:
new=udata2.pivot(
    index='user id',
    columns='item id',
    values='rating'
)#.fillna(0)

In [12]:
new.head(5)

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# first approach: corr matrix

In [ ]:
def corr_movie():
  movie=input('enter your movie:')
  print('AH good taste ;) ')

  #make it better
  #id=uitem2[uitem2['movietitle'].str.contains(movie)]['item id'][0]
  a= uitem2[uitem2['movietitle'].str.contains(movie)]
  if len(a)>1:
    print('\n which one?')
    print(tabulate(a, headers = 'keys', tablefmt = 'psql'))
    id= int( input('type the item id!:') )
  else:
    id= a['item id'][0]
  
  #corr
  new2=new.fillna(0)
  corr= new2.corrwith(new2[id])
  b= pd.DataFrame({'item id':corr.index,'corr':corr.values},index=list(corr.index))
  final= pd.merge(uitem2,b,on="item id")
  final=final[['movietitle', 'corr']].sort_values('corr',ascending=False)[1:]

  print('\n the similar ones are:')
  print(tabulate(final[:5], headers = 'keys', tablefmt = 'psql'))

corr_movie()

enter your movie:Aladdin
AH good taste ;) 

 which one?
+-----+-----------+----------------------------------------+
|     |   item id | movietitle                             |
|-----+-----------+----------------------------------------|
|  94 |        95 | Aladdin (1992)                         |
| 421 |       422 | Aladdin and the King of Thieves (1996) |
+-----+-----------+----------------------------------------+
type the item id!:95

 the similar ones are:
+-----+----------------------------------------+----------+
|     | movietitle                             |     corr |
|-----+----------------------------------------+----------|
|  70 | Lion King, The (1994)                  | 0.683855 |
| 587 | Beauty and the Beast (1991)            | 0.650583 |
|  81 | Jurassic Park (1993)                   | 0.568547 |
| 203 | Back to the Future (1985)              | 0.545456 |
|  98 | Snow White and the Seven Dwarfs (1937) | 0.539332 |
+-----+----------------------------------------+-----

# 2nd approach: KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30, n_jobs=-1)

In [ ]:
uitem3= uitem[['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western']]

In [ ]:
# new2=new.fillna(0)
from scipy.sparse import csr_matrix
csrnew2=csr_matrix(uitem3)
model_knn.fit(csrnew2)


In [98]:
def knn():
  movie=input('enter your movie:')
  print('AH good taste ;) ')

  '''
  model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30, n_jobs=-1)
  csrnew2=csr_matrix(uitem3)
  model_knn.fit(csrnew2)
  '''
  #make it better
  a= uitem2[uitem2['movietitle'].str.contains(movie)]
  if len(a)>1:
    print('\n which one?')
    print(tabulate(a, headers = 'keys', tablefmt = 'psql'))
    id= int( input('type the item id!:') )-1
  else:
    id= a['item id'][0]-1
  
  #simi
  new2=new.fillna(0)
  corr= model_knn.kneighbors(
            [list(uitem3.iloc[id])],
            5+1,return_distance=True)
  b= pd.DataFrame({'item id':corr[1][0]+1,'simi':corr[0][0]},index=list(corr[1][0]))
  final= pd.merge(uitem2,b,on="item id")
  final= final[['movietitle', 'simi']].sort_values('simi',ascending=False)[:-1]

  print('\n the similar ones are:')
  print(tabulate(final, headers = 'keys', tablefmt = 'psql'))

knn()

enter your movie:Aladdin
AH good taste ;) 

 which one?
+-----+-----------+----------------------------------------+
|     |   item id | movietitle                             |
|-----+-----------+----------------------------------------|
|  94 |        95 | Aladdin (1992)                         |
| 421 |       422 | Aladdin and the King of Thieves (1996) |
+-----+-----------+----------------------------------------+
type the item id!:95

 the similar ones are:
+----+----------------------------------------+----------+
|    | movietitle                             |     simi |
|----+----------------------------------------+----------|
|  0 | Toy Story (1995)                       | 0.133975 |
|  2 | Snow White and the Seven Dwarfs (1937) | 0.133975 |
|  3 | All Dogs Go to Heaven 2 (1996)         | 0.133975 |
|  4 | Hunchback of Notre Dame, The (1996)    | 0.133975 |
|  5 | Hercules (1997)                        | 0.105573 |
+----+----------------------------------------+----------+
